In [ ]:
#HSI Table
HSI_Database = 'HSIFROM2019_copy'
#Start Date & End Date

#Graph Title
Graph_Title = "With Wu As Change Year (2024-2025)"
#File Title

# Setup and Download Source code


In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'stock8word' # Project ID inserted based on the query results selected to explore
location = 'asia-southeast1' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf four-pillars
!rm -rf fourpillars
!git clone https://github.com/gazaay/four-pillars.git
!mv four-pillars fourpillars
!ls
!pwd
!cd fourpillars && git pull
!ls
!pip install LunarCalendar
# install mpl-tc-fonts
!pip install git+https://github.com/Hsins/mpl-tc-fonts.git
!pip install mplfinance
!pip install --upgrade mplfinance

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'four-pillars'...
remote: Enumerating objects: 2321, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 2321 (delta 49), reused 117 (delta 26), pack-reused 2178 (from 1)
Receiving objects: 100% (2321/2321), 108.85 MiB | 4.87 MiB/s, done.
Resolving deltas: 100% (574/574), done.
Updating files: 100% (2205/2205), done.
app.log  drive	fourpillars  sample_data
/content
Already up to date.
app.log  drive	fourpillars  sample_data
  Cloning https://github.com/Hsins/mpl-tc-fonts.git to /tmp/pip-req-build-d7ehv_lx
  Running command git clone --filter=blob:none --quiet https://github.com/Hsins/mpl-tc-fonts.git /tmp/pip-req-build-d7ehv_lx
  Resolved https://github.com/Hsins/mpl-tc-fonts.git to commit 0cc159a2dbf1eb199f0bd847eabfee75403346b6
  Preparing metadata (setup.py) ... done


# Overall planning
1. get stock birthday with finance.get_listing_date_timestamp
2. get stock 8w with bazi.get_heavenly_branch_ymdh_pillars_base to base_8w
3. Define a time range star date today minus 725 days, end date today
4. create a blank data frame, create a "time" column where each element are incremented by 2hours with the time range and add future 12 months
5. use adding_8w_pillars to add pillars in and put base_8w into the dataset
6. reusing the timerange - get stock history data with finance.get_historical_data_UTC(hong_kong_stocks[0], start_date=(current day minus 725 days), end_date=(current day), interval='1h')
7. merge the history data with the dataset where time is the key.
7. get stock chengseng with create_chengseng_for_dataset
8. feature engineering
9. split data sets for random forest training
10. train random forest
11. predict with random forest model with the future 12 months

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from fourpillars.app  import finance  # Assume you have finance module with functions like get_listing_date_timestamp and get_historical_data_UTC
from fourpillars.app  import bazi # Assume you have bazi module with functions like get_heavenly_branch_ymdh_pillars_base, adding_8w_pillars, create_chengseng_for_dataset
from fourpillars.app import chengseng
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [ ]:
import random, string

TOKEN_RUNTIME = random.choice(string.ascii_letters + string.digits)

In [ ]:
# @title Step 1: Get stock birthday
from fourpillars.app  import finance

symbol = '00700'  # Replace with your stock symbol
symbol_code = "0700.hk"

symbol = '^HSI'
symbol_code = "^HSI"

try:
  stock_birthday_timestamp = finance.get_listing_date_timestamp(symbol)
except Exception as ex:
  print(f"Error: During collect birthday {ex}")
  stock_birthday_timestamp = datetime(1969,11,24,9,30,0)

stock_birthday_timestamp = datetime(1969,11,24,9,30,0)
subject_birthday_timestamp = datetime(1871,6,27,9,30,0)
subject_birthday_timestamp = datetime(1948,9,9,7,30,0)
print(stock_birthday_timestamp)

Listing Date not found on the webpage.
1969-11-24 09:30:00


In [ ]:
import pandas as pd
import pytz

def convert_timestamp_to_GMT8(df):
    # Assuming 'timestamp_column' is your timestamp column
    # Convert it to datetime type
    df['time'] = pd.to_datetime(df['time'])

    # Define the UTC timezone
    utc = pytz.timezone('UTC')

    # Define the GMT+8 timezone
    gmt_plus_8 = pytz.timezone('Asia/Shanghai')  # Adjust timezone as needed

    # Localize the timestamp column to UTC
    # df['time'] = df['time'].apply(lambda x: utc.localize(x))

    # Convert the timestamp to GMT+8
    df['time'] = df['time'].dt.tz_convert(gmt_plus_8)

    return df

# Define your SQL query
# sql_query = """
# SELECT *
# FROM stock8word.HSI8.HSI_2017
# """
sql_query = f"""
SELECT *
FROM stock8word.HSI8.{HSI_Database}
"""


# Execute the query and convert results to a Pandas DataFrame
query_job = client.query(sql_query)
results = query_job.to_dataframe()


results = convert_timestamp_to_GMT8(results)

results

results[['time','Open', 'High', 'Low', 'Close']] = results[['time','open', 'high', 'low', 'close']]

results_from_GCP = results[['time','Open', 'High', 'Low', 'Close']]


In [ ]:
results_from_GCP.columns.tolist()

['time', 'Open', 'High', 'Low', 'Close']

In [ ]:
symbol = '00001'  # Replace with your stock symbol
symbol_code = "0001.hk"

try:
  temp_bd = finance.get_listing_date_timestamp(symbol)
except Exception as ex:
  print(f"Error: During collect birthday {ex}")
  temp_bd = datetime(1969,11,24,9,30,0)

print(temp_bd)

Listing Date not found on the webpage.
None


In [ ]:
# @title Step 2: Get stock 8w
base_8w = bazi.get_heavenly_branch_ymdh_pillars_current(stock_birthday_timestamp.year,
                                                            stock_birthday_timestamp.month,
                                                            stock_birthday_timestamp.day,
                                                            stock_birthday_timestamp.hour)

base_subject_8w = bazi.get_heavenly_branch_ymdh_pillars_current(subject_birthday_timestamp.year,
                                                            subject_birthday_timestamp.month,
                                                            subject_birthday_timestamp.day,
                                                            subject_birthday_timestamp.hour)


# # @title Step 2: Get stock 8w
# base_8w = bazi.get_heavenly_branch_ymdh_pillars_current_flip_Option_2(stock_birthday_timestamp.year,
#                                                             stock_birthday_timestamp.month,
#                                                             stock_birthday_timestamp.day,
#                                                             stock_birthday_timestamp.hour)

# base_subject_8w = bazi.get_heavenly_branch_ymdh_pillars_current_flip_Option_2(subject_birthday_timestamp.year,
#                                                             subject_birthday_timestamp.month,
#                                                             subject_birthday_timestamp.day,
#                                                             subject_birthday_timestamp.hour)


In [ ]:
# @title Step 3: Define time range
today = datetime.now()
# Set the time to 9:30 AM
today = today.replace(hour=9, minute=00, second=0, microsecond=0)

start_date = today - timedelta(days=1525)
end_date = today + timedelta(days=220)
# @title Step 4: Create a blank data frame with time column
time_range = pd.date_range(start=start_date, end=end_date, freq='1H').union(pd.date_range(end_date, end_date + pd.DateOffset(months=12), freq='D'))
dataset = pd.DataFrame({'time': time_range})
# @title Step 5: Adding 8w pillars to the dataset
dataset = chengseng.adding_8w_pillars(dataset)

In [ ]:
# @title Step 6: Get stock history data
symbol_code = '^HSI'
historical_data = finance.get_historical_data_UTC(symbol_code, start_date=start_date, end_date=end_date, interval='1h')


In [ ]:
historical_data

In [ ]:
results_cutoff = results_from_GCP.copy()

# Filter out rows where 'time' is older than 200 days from today
today = pd.Timestamp.now(tz='Asia/Shanghai')
cutoff_date = today - pd.Timedelta(days=200)
results_cutoff = results_cutoff[results['time'] < cutoff_date]

historical_data = results_from_GCP

# import pandas as pd

# # Assuming you have a DataFrame called Historical_data
# # Let's say you want to remove rows where Date > certain_datetime

# certain_datetime = pd.to_datetime('2023-11-10 12:00:00+08:00')  # Replace this with your desired datetime

# # Convert 'Date' column to datetime
# historical_data['Date'] = pd.to_datetime(historical_data['Date'])

# # Boolean indexing to filter rows
# historical_data_filtered = historical_data[historical_data['Date'] <= certain_datetime]

# # Historical_data_filtered now contains only the rows where Date is less than or equal to certain_datetime

# historical_data = historical_data_filtered


In [ ]:
print(base_8w.columns.tolist())

In [ ]:
# Set the timezone to 'Asia/Hong_Kong'
dataset['time'] = dataset['time'].dt.tz_localize('Asia/Hong_Kong')
dataset['本時'] = base_8w["時"]
dataset['本日'] = base_8w["日"]
dataset['-本時'] = base_8w["-時"]
dataset['本月'] = base_8w["月"]
dataset['本年'] = base_8w["年"]
dataset['-本月'] = base_8w["-月"]
dataset['合年']= base_8w["合年"]
dataset['合月']= base_8w["合月"]
dataset['合日']= base_8w["合日"]
dataset['合時']= base_8w["合時"]
        # "合年": resolveHeavenlyStem(flipped_year_stem) + resolveEarthlyBranch(flipped_year_branch),
        # "合月": resolveHeavenlyStem(flipped_month_stem) + resolveEarthlyBranch(flipped_month_branch),
        # "合日": resolveHeavenlyStem(flipped_day_stem) + resolveEarthlyBranch(flipped_day_branch),
        # "合時": resolveHeavenlyStem(flipped_hour_stem) + resolveEarthlyBranch(flipped_hour_branch),
# dataset['time'] = dataset['time'].dt.tz_localize('Asia/Hong_Kong')
# dataset['subject_本時'] = base_subject_8w["時"]
# dataset['subject_本日'] = base_subject_8w["日"]
# dataset['subject_-本時'] = base_subject_8w["-時"]
# dataset['subject_本月'] = base_subject_8w["月"]
# dataset['subject_本年'] = base_subject_8w["年"]
# dataset['subject_-本月'] = base_subject_8w["-月"]

dataset

In [ ]:
print(historical_data["time"].iloc[0])

print(dataset['time'].iloc[0])

historical_data.set_index('time', inplace=True)


In [ ]:
historical_data

In [ ]:
# @title Step 7: Merge history data with the dataset
dataset = pd.merge(dataset, historical_data, left_on='time', right_index=True, how='left')



In [ ]:
dataset.columns.to_list()

In [ ]:
# @title Step 7 (cont.): Get stock chengseng and merge with the dataset

dataset = chengseng.create_chengseng_for_dataset(dataset)
# dataset = pd.merge(dataset, chengseng_data, on='time', how='left')

In [ ]:
dataset

In [ ]:
# from app.haap import add_haap_features_to_df

# dataset = add_haap_features_to_df(dataset)

print(f"{dataset.columns.tolist()}")
dataset.to_csv(f"drive/MyDrive/01_888/Processing/Ken_{TOKEN_RUNTIME}_before_training.csv", index=False)

In [ ]:
categorical_features = dataset.columns.tolist()

In [ ]:
categorical_features

In [ ]:
# Define your custom encoding mapping
custom_encoding_mapping = {
    "甲子": 1, "乙丑": 2, "丙寅": 3, "丁卯": 4, "戊辰": 5, "己巳": 6, "庚午": 7, "辛未": 8,
    "壬申": 9, "癸酉": 10, "甲戌": 11, "乙亥": 12, "丙子": 13, "丁丑": 14, "戊寅": 15, "己卯": 16,
    "庚辰": 17, "辛巳": 18, "壬午": 19, "癸未": 20, "甲申": 21, "乙酉": 22, "丙戌": 23, "丁亥": 24,
    "戊子": 25, "己丑": 26, "庚寅": 27, "辛卯": 28, "壬辰": 29, "癸巳": 30, "甲午": 31, "乙未": 32,
    "丙申": 33, "丁酉": 34, "戊戌": 35, "己亥": 36, "庚子": 37, "辛丑": 38, "壬寅": 39, "癸卯": 40,
    "甲辰": 41, "乙巳": 42, "丙午": 43, "丁未": 44, "戊申": 45, "己酉": 46, "庚戌": 47, "辛亥": 48,
    "壬子": 49, "癸丑": 50, "甲寅": 51, "乙卯": 52, "丙辰": 53, "丁巳": 54, "戊午": 55, "己未": 56,
    "庚申": 57, "辛酉": 58, "壬戌": 59, "癸亥": 60,
    "甲": 201, "乙": 202, "丙": 203, "丁": 204, "戊": 205,
    "己": 206, "庚": 207, "辛": 208, "壬": 209, "癸": 210,
    "傷官": -310, "七殺": -308, "劫財": -306, "比肩": -304,
    "食神": -302, "偏印": 302, "偏財": 304, "正財": 306,
    "正官": 308, "正印": 310,
    "長生": 402, "沐浴": -402, "冠帶": 406, "臨官": 408,
    "帝旺": 410, "衰": -404, "病": -406, "死": -410,
    "墓庫": -404, "絕": -408, "胎": 400, "養": 400,
    "子": 111, "丑": 112, "寅": 101, "卯": 102,
    "辰": 103, "巳": 104, "午": 105, "未": 106,
    "申": 107, "酉": 108, "戌": 109, "亥": 110
}

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import datetime

label_encoders ={}
columns_to_ignore = ['time', 'Open_x', 'High_x', "Low_x", "Close_x", "Volume_x" ,'Open_y',
 'High_y',
 'Low_y',
 'Close_y',
 'Volume_y',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',]

def process_encode_data(data_, categorical_features):
    data = data_.copy()
    for feature in categorical_features:
          if feature in columns_to_ignore:
            continue
          try:
            # Initialize the LabelEncoder with your custom mapping
            label_encoder = LabelEncoder()
            label_encoder.fit(list(custom_encoding_mapping.values()))
            encoded_values = None

            try:
              # Try to map the values using custom_encoding_mapping
              encoded_values = data[feature].map(custom_encoding_mapping)
            except Exception as e:
              print("An error occurred:", e)
            # Assuming 'data' is your DataFrame and 'feature' is the column containing the large numbers
            formatted_values = []

            # If value in feature could be int or float. We will add it there. if it is not, we will
            # add the raw value instead.
            for value in data[feature]:
              if isinstance(value, (int, float)):
                  formatted_values.append(f'{value:.0f}')
              else:
                  formatted_values.append(value)


            data[feature] = formatted_values


            # Convert the column to numeric with errors='coerce' to convert non-numeric to NaN
            numeric_data = pd.to_numeric(data[feature], errors='coerce')

            if numeric_data.isnull().any():
              # Check if any value is not found in the custom_encoding_mapping
              if encoded_values.isnull().any() :
                  # Fallback to using fit_transform
                  print(feature + " will fall back to use fit_transform")
                  data[feature] = label_encoder.fit_transform(data[feature])
              else:
                  # Use transform with your custom mapping to encode the data
                  data[feature] = label_encoder.transform(data[feature].map(custom_encoding_mapping))

              label_encoders[feature] = label_encoder
            else:
              print(f"Column {feature} doesn't need to encode")
              data[feature] = data[feature].astype(float)


            print(f"Unique values for {feature}: {data[feature].unique()}")
            #print(f"decode the value {label_encoders[feature].inverse_transform(data[feature])}")
          except Exception as e:
            print("An error occurred:", e)
            continue
    return data, label_encoder


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
# @title Step 8: Feature engineering (assuming you have additional features to engineer)


data_with_custom_features, encoder = process_encode_data(dataset, categorical_features)

# Create a copy of the 'Close' column shifted 90 days into the future
# data_with_custom_features['last_90_days_close'] = data_with_custom_features['Close'].copy().dropna().shift(-90)

# Fill missing values in the 'last_90_days_close' column with the current value
# data_with_custom_features['last_90_days_close'] = data_with_custom_features['last_90_days_close'].fillna(method='ffill')

data_with_custom_features.to_csv(f"drive/MyDrive/01_888/Processing/Feature_Engineering_Encode_with_Haap_{TOKEN_RUNTIME}_before_training.csv",)


In [ ]:
# Check if there are any missing values in data_with_custom_features
has_missing_values = data_with_custom_features.isna().any().any()

# Alternatively, you can use isnull()
# has_missing_values = data_with_custom_features.isnull().any().any()

if has_missing_values:
    print("data_with_custom_features contains missing values.")
else:
    print("data_with_custom_features does not contain missing values.")


In [ ]:
data_with_custom_features

In [ ]:
# @title Step 9: Split datasets for random forest training

# data_with_custom_features = data_with_custom_features.dropna(subset = ['time', 'Open', 'High', "Low", "Close", "Volume" ,])

data_with_custom_features_for_training = data_with_custom_features.dropna(subset=['time', 'Open', 'High', 'Low', 'Close',
                                                                                  # 'Volume'
                                                                                    ])

results_cutoff = data_with_custom_features_for_training.copy()

# Filter out rows where 'time' is older than 200 days from today
today_ = pd.Timestamp.now(tz='Asia/Shanghai')
cutoff_date = today_ - pd.Timedelta(days=200)
# results_cutoff = results_cutoff[results_cutoff['time'] < cutoff_date]

features = results_cutoff.drop(['time', 'Open', 'High', "Low", "Close",
                                                        # "Volume",
                                                        # '流時',
                                                        # '流日',
                                                        # '-流時',
                                                        # '流月',
                                                        # '流年',
                                                        # '-流月',
                                                        '本時',
                                                        '本日',
                                                        '-本時',
                                                        '本月',
                                                        '本年',
                                                        '-本月'
                                                         ], axis=1)  # Adjust based on your actual features
# labels = dataset['label']  # Adjust based on your actual labels

labels = data_with_custom_features_for_training[['Open', 'High', "Low", "Close",
                                                #  "Volume"
 ]]  # Adjust based on your actual labels

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)



In [ ]:
data_with_custom_features.columns.tolist()

In [ ]:
# Example: Check data types and NaN values
print("#########XXXXXXXX#############")
print(X_train.dtypes)
print("#########YYYYYYYY#############")
print(y_train.dtypes)

# Check for NaN values
print("#########XXXXXXXX#############")
print(X_train.isna().sum())
print("#########YYYYYYYY#############")
print(y_train.isna().sum())

In [ ]:
# for subject
# base_subject_8w

# dataset['time'] = dataset['time'].dt.tz_localize('Asia/Hong_Kong')
# dataset['subject_本時'] = base_subject_8w["時"]
# dataset['subject_本日'] = base_subject_8w["日"]
# dataset['subject_-本時'] = base_subject_8w["-時"]
# dataset['subject_本月'] = base_subject_8w["月"]
# dataset['subject_本年'] = base_subject_8w["年"]
# dataset['subject_-本月'] = base_subject_8w["-月"]

#Time range to predict
# get current time  8w pillar
# chengsengfor time range
# happ for time range
# create features
# process encoding
# encoded custom features
# save to csv
# fill in the gaps
# remove na
# training sets.
# split sets
# training
# test predict


# get current time  8w pillar
# chengsengfor time range
# happ for time range
# create features
# process encoding
# encoded custom features
# prediction

In [ ]:
# @title Step 10: Train random forest
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
mor_model = MultiOutputRegressor(RandomForestRegressor())

mor_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

predictions = mor_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error (MSE): {mse}')
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print(f'R-squared (R2 Score): {r2}')
from sklearn.metrics import explained_variance_score
explained_var = explained_variance_score(y_test, predictions)
print(f'Explained Variance Score: {explained_var}')
from sklearn.metrics import median_absolute_error
medae = median_absolute_error(y_test, predictions)
print(f'Median Absolute Error (MedAE): {medae}')

# Assuming X_train is your training data and has the feature names
feature_names = X_train.columns

# Check feature importance
feature_importance = mor_model.estimators_[0].feature_importances_

# Combine feature names with importances and sort by importance
sorted_features = sorted(zip(feature_names, feature_importance), key=lambda x: x[1], reverse=True)

# Print sorted feature importance
print('Sorted Feature Importance:')
for i, (feature_name, importance) in enumerate(sorted_features):
    print(f'Rank {i+1}: Feature {feature_name}, Importance: {importance}')


In [ ]:
# !pip install shap

In [ ]:
# Get today's date and time at 9:30 AM
import pandas as pd
from datetime import datetime, timedelta

today_930 = datetime.now().replace(hour=9, minute=30, second=0, microsecond=0)
# Convert today_930 to 'Asia/Hong_Kong' timezone
today_930 = pd.Timestamp(today_930).tz_localize('Asia/Hong_Kong')

# Filter rows where 'time' is greater than today at 9:30 AM
future_features = data_with_custom_features.drop(['time', 'Open', 'High', "Low", "Close",
                                                  # "Volume" ,
                                                        # '流時',
                                                        # '流日',
                                                        # '-流時',
                                                        # '流月',
                                                        # '流年',
                                                        # '-流月',
                                                        '本時',
                                                        '本日',
                                                        '-本時',
                                                        '本月',
                                                        '本年',
                                                        '-本月'], axis=1)

# # Add any additional feature engineering steps if needed

# future_features['last_90_days_close'] = data_with_custom_features['Close'].dropna().rolling(90).mean()

# # Drop rows where 'last_90_days_close' is NaN
# future_features.dropna(subset=['last_90_days_close'], inplace=True)

# Make predictions
future_predictions = mor_model.predict(future_features)

In [ ]:
# import pandas as pd
# from datetime import datetime, timedelta

# # Get today's date and time at 9:30 AM
# today_930 = datetime.now().replace(hour=9, minute=30, second=0, microsecond=0)
# # Convert today_930 to 'Asia/Hong_Kong' timezone
# today_930 = pd.Timestamp(today_930).tz_localize('Asia/Hong_Kong')

# # Define yesterday and tomorrow at the same time as today_930
# yesterday_930 = today_930 - timedelta(days=1)
# tomorrow_930 = today_930 + timedelta(days=1)

# # Filter data to include only yesterday, today, and tomorrow
# mask = (data_with_custom_features['time'] >= yesterday_930) & (data_with_custom_features['time'] <= tomorrow_930)
# filtered_data = data_with_custom_features[mask]

# # Drop unnecessary columns
# filtered_features_s = filtered_data.drop(['time', 'Open', 'High', "Low", "Close", '本時', '本日', '-本時', '本月', '本年', '-本月',
#                                           # 'Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close'
# ], axis=1)

# # Make predictions
# predictions = mor_model.predict(filtered_features_s)
# filtered_data_c = filtered_data.copy()
# # Assuming predictions returns a DataFrame or needs reshaping if not
# filtered_data_c[['Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close']] = predictions

# # Print predictions
# print(predictions)

# # Report on prediction outcomes
# for i, row in filtered_data_c.iterrows():
#     reason = "This prediction was made because..."  # Placeholder for actual logic
#     print(f"Prediction for index {i}: {row['time']} {row['Predicted_Close']}, Reason: {reason}")


In [ ]:
# # @title Step 11: Try to understand estimation with shap

# import shap

# future_features_for_shap = data_with_custom_features.copy()
# future_features_for_shap = future_features_for_shap[(future_features_for_shap['time'] >= '2024-02-23') & (future_features_for_shap['time'] <= '2024-02-25')]

# # Assuming future_features contain a column named 'date' representing dates
# # Filter future_features for the time period between Feb 23 and Feb 25
# filtered_future_features = future_features_for_shap.drop(['time', 'Open', 'High', "Low", "Close",
#                                                   # "Volume" ,
#                                                         # '流時',
#                                                         # '流日',
#                                                         # '-流時',
#                                                         # '流月',
#                                                         # '流年',
#                                                         # '-流月',
#                                                         '本時',
#                                                         '本日',
#                                                         '-本時',
#                                                         '本月',
#                                                         '本年',
#                                                         '-本月', 'Predicted_Close', 'Predicted_High', 'Predicted_Low', 'Predicted_Open'], axis=1)

# prediction = mor_model.predict(filtered_future_features)

# # Initialize TreeExplainer
# explainer = shap.TreeExplainer(mor_model)

# # Calculate SHAP values for the chosen instance
# shap_values = explainer.shap_values(filtered_future_features)

# # Visualize the explanation
# shap.initjs()
# shap.force_plot(explainer.expected_value, shap_values, filtered_future_features)


In [ ]:
data_with_custom_features[['Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close']] = future_predictions
# data_with_custom_features[['Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close', 'Predicted_Volume']] = future_predictions
data_for_web = pd.DataFrame()

data_for_web[['time', 'Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close']]= data_with_custom_features[['time', 'Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close']]
data_for_web['Predicted_Volume']=0
data_for_web.to_csv(f"drive/MyDrive/01_888/Prediction/Prediction_{today}_version.csv")
# Display the predictions for the future 12 months
print("Predictions for the Future 12 Months:")
print(data_with_custom_features)

In [ ]:
future_features.columns.tolist()

In [ ]:
!pip install git+https://github.com/Hsins/mpl-tc-fonts.git

In [ ]:
# .tz_localize('Asia/Hong_Kong')
selected_columns = ['time', 'Close', 'Predicted_Close']
selected_data = data_with_custom_features[selected_columns]
# selected_data = selected_data.dropna(subset=['Close'])



In [ ]:
print(selected_data)
selected_data.to_csv(f"drive/MyDrive/01_888/Processing/Predictions_Option_1_{TOKEN_RUNTIME}_before_training.csv",)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample DataFrame

df = selected_data
# df = pd.DataFrame(data)

# Ensure time column is in datetime format
df['time'] = pd.to_datetime(df['time'])

# Filter DataFrame to only include dates from 2024 to 2025
df_filtered = df[(df['time'] >= '2024-01-01') & (df['time'] <= '2025-12-31')]

# Plotting
plt.figure(figsize=(82, 6))
# plt.plot(df_filtered['time'], df_filtered['Close'], label='Actual Close', marker='o')
plt.plot(df_filtered['time'], df_filtered['Predicted_Close'], label='Predicted Close', marker='.')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.title(f'Predicted Close Prices {Graph_Title}')
plt.legend()
plt.grid(True)

# Set x-axis to show monthly ticks
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(nbins=95))  # Show every month
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))

# Rotate x-axis labels for better readability
plt.xticks(rotation=85)
plt.tight_layout()

plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_Master__{Graph_Title}_{end_date}_version_1.pdf", format="pdf")


# Show plot
plt.show()


In [ ]:
# Create a reverse mapping
reverse_encoding_mapping = {v: k for k, v in custom_encoding_mapping.items()}
encoded_value = 63
decoded_value = reverse_encoding_mapping.get(encoded_value, f"Unknown Encoding: {encoded_value}")
print(f"Decoded value: {decoded_value}")

# next(key for key, value in custom_encoding_mapping.items() if value == label_encoders[0].inverse_transform([26]))

In [ ]:
# Assuming data_with_custom_features is your DataFrame
row_data = data_with_custom_features.iloc[0].to_dict()

# Now, print the full row of data
print(row_data)

In [ ]:
data_with_custom_features


In [ ]:
from datetime import datetime

today = datetime.now()
# Set the time to 9:30 AM
today = today.replace(hour=9, minute=00, second=0, microsecond=0)

In [ ]:
# df = data_with_custom_features.copy()
# df['流月'] = label_encoders['流月'].inverse_transform(df['流月'])
# df['流月']

In [ ]:
    # # Define start and last dates for filtering
    # df = data_with_custom_features.copy()
    # today = pd.Timestamp('now', tz='UTC')  # Ensures 'today' is in UTC to match your time column
    # start_date = today - timedelta(days=15)  # Today minus 15 days
    # last_date = start_date + timedelta(days=90)  # Start date plus 8 months (approximated with 30 days per month)

    # # Filter data between start date and last date
    # filtered_df = df[(df['time'] >= start_date) & (df['time'] <= last_date)]

    # # Decode the '流月' using the label encoder for '流月'
    # df['流月'] = label_encoders['流月'].inverse_transform(df['流月'])

    # # Group by '流月' and aggregate the necessary statistics
    # results = {}
    # grouped = filtered_df.groupby('流月')

    # for name, group in grouped:
    #   print(f"This is the group { name}")
    #   # print(group)

In [ ]:
# import pandas as pd
# import json
# from datetime import datetime, timedelta
# from sklearn.preprocessing import LabelEncoder

# def read_data():
#     # Load your data from a CSV file or another source
#     df = data_with_custom_features.copy()
#     return df

# def calculate_percentage_change(open_price, close_price):
#     if open_price == 0:
#         return 0  # Prevent division by zero
#     return ((close_price - open_price) / open_price) * 100

# def process_data(df, custom_encoding_mapping):
#     # Ensure 'time' is handled as UTC datetime
#     df['time'] = pd.to_datetime(df['time'], utc=True)
#     df.sort_values('time', inplace=True)  # Sort by time
#     # Define start and last dates for filtering
#     today = pd.Timestamp('now', tz='UTC')  # Ensures 'today' is in UTC to match your time column
#     start_date = today - timedelta(days=15)  # Today minus 15 days
#     last_date = start_date + timedelta(days=90)  # Start date plus 8 months (approximated with 30 days per month)

#     # Filter data between start date and last date
#     filtered_df = df[(df['time'] >= start_date) & (df['time'] <= last_date)]
#     # Decode the '流月'
#     df['流月'] = label_encoders['流月'].inverse_transform(df['流月'])

#     # Group by '流月' and process
#     results = {}
#     grouped = filtered_df.groupby('流月')
#     next_month = []
#     for name, group in grouped:
#         # Aggregate to get a single value for each predictive measure
#         aggregated_data = {
#             'date': group['time'].iloc[0].strftime('%Y-%m-%d'),
#             'open': group['Predicted_Open'].iloc[0],  # First value in the group
#             'close': group['Predicted_Close'].iloc[-1],  # Last value in the group
#             'high': group['Predicted_High'].max(),  # Maximum value in the group
#             'low': group['Predicted_Low'].min(),  # Minimum value in the group
#             'change': calculate_percentage_change(group['Predicted_Open'].iloc[0], group['Predicted_Close'].iloc[-1])  # Change from open to close
#         }
#         next_month.append(aggregated_data)
#         # results[name] = [aggregated_data]  # Store results for this group

#     return next_month

# def process_weekly_data(df, custom_encoding_mapping):
#     # Ensure 'time' is handled as UTC datetime
#     df['time'] = pd.to_datetime(df['time'], utc=True)
#     df.sort_values('time', inplace=True)  # Sort by time
#     # Define start and last dates for filtering
#     today = pd.Timestamp('now', tz='UTC')  # Ensures 'today' is in UTC to match your time column
#     start_date = today - timedelta(days=15)  # Today minus 15 days
#     last_date = start_date + timedelta(days=90)  # Start date plus 8 months (approximated with 30 days per month)

#     # Filter data between start date and last date
#     filtered_df = df[(df['time'] >= start_date) & (df['time'] <= last_date)]
#     # Decode the '流月'
#     df['流月'] = label_encoders['流月'].inverse_transform(df['流月'])

#     # Group by week and process
#     weekly_data = []
#     grouped = filtered_df.set_index('time').resample('W').agg({
#         'Predicted Open': 'first',
#         'Predicted Close': 'last',
#         'Predicted High': 'max',
#         'Predicted Low': 'min'
#     })

#     for week_start, group in grouped.iterrows():
#         if not group.isnull().any():  # Check for any NaN values which can occur if no data is present for the week
#             aggregated_data = {
#                 'date': week_start.strftime('%Y-%m-%d'),
#                 'open': group['Predicted Open'],
#                 'close': group['Predicted Close'],
#                 'high': group['Predicted High'],
#                 'low': group['Predicted Low'],
#                 'change': f"{calculate_percentage_change(group['Predicted Open'], group['Predicted Close']):.2f}%"
#             }
#             weekly_data.append(aggregated_data)

#     return weekly_data

# def process_year_data(df, custom_encoding_mapping):
#     # Ensure 'time' is handled as UTC datetime
#     df['time'] = pd.to_datetime(df['time'], utc=True)
#     df.sort_values('time', inplace=True)  # Sort by time

#     # Define start and last dates for filtering
#     today = pd.Timestamp('now', tz='UTC')  # Ensures 'today' is in UTC to match your time column
#     start_date = today - timedelta(days=205)  # Today minus 15 days
#     last_date = start_date + timedelta(days=900)  # Start date plus 8 months (approximated with 30 days per month)

#     # Filter data between start date and last date
#     filtered_df = df[(df['time'] >= start_date) & (df['time'] <= last_date)]
#     # Decode the '流月'
#     df['流年'] = label_encoders['流年'].inverse_transform(df['流年'])

#     # Group by '流月' and process
#     results = {}
#     grouped = filtered_df.groupby('流年')
#     next_month = []
#     for name, group in grouped:
#         # Aggregate to get a single value for each predictive measure
#         aggregated_data = {
#             'name': group['time'].iloc[0].strftime('%Y-%m-%d'),
#             # 'open': group['Predicted_Open'].iloc[0],  # First value in the group
#             'value': group['Predicted_Close'].iloc[-1]
#         }
#         next_month.append(aggregated_data)
#         # results[name] = [aggregated_data]  # Store results for this group

#     return next_month


# def process_day_data(df, custom_encoding_mapping):
#     # Ensure 'time' is handled as UTC datetime
#     df['time'] = pd.to_datetime(df['time'], utc=True)
#     df.sort_values('time', inplace=True)  # Sort by time
#     # Define start and last dates for filtering
#     today = pd.Timestamp('now', tz='UTC')  # Ensures 'today' is in UTC to match your time column
#     start_date = today - timedelta(days=3)  # Today minus 15 days
#     last_date = start_date + timedelta(days=3)  # Start date plus 8 months (approximated with 30 days per month)

#     # Filter data between start date and last date
#     filtered_df = df[(df['time'] >= start_date) & (df['time'] <= last_date)]
#     # Decode the '流月'
#     df['流日'] = label_encoders['流日'].inverse_transform(df['流日'])

#     # Group by '流月' and process
#     results = {}
#     grouped = filtered_df.groupby('流日')
#     next_month = []
#     for name, group in grouped:
#         # Aggregate to get a single value for each predictive measure
#         aggregated_data = {
#             'date': f"{group['time'].iloc[0]} with { calculate_percentage_change(group['Predicted_Open'].iloc[0], group['Predicted_Close'].iloc[-1])}",
#             'open': group['Predicted_Open'].iloc[0],  # First value in the group
#             'close': group['Predicted_Close'].iloc[-1],  # Last value in the group
#             'high': group['Predicted_High'].max(),  # Maximum value in the group
#             'low': group['Predicted_Low'].min(),  # Minimum value in the group
#         }
#         next_month.append(aggregated_data)
#         # results[name] = [aggregated_data]  # Store results for this group

#     return next_month


# def output_json(data):
#     # Convert the data to JSON format with indentation for readability
#     json_data = json.dumps(data, indent=4, default=str)
#     return json_data

# def main():
#     # Assume label_encoder and custom_encoding_mapping are defined here
#     # label_encoder = LabelEncoder()  # Normally you would fit this to your data beforehand
#     custom_encoding_mapping = {'your_key': 'your_value'}  # Populate with your actual mapping
#     print ("Start processing data for Month by month grouping. ")
#     df = read_data()
#     processed_data = process_data(df, custom_encoding_mapping)
#     json_output = output_json(processed_data)
#     print(json_output)

#     print ("Start processing data for Weekly grouping. ")
#     processed_data = process_weekly_data(df, custom_encoding_mapping)
#     json_output = output_json(processed_data)
#     print(json_output)

#     print ("Start processing data for Yearly grouping. ")
#     processed_data = process_year_data(df, custom_encoding_mapping)
#     json_output = output_json(processed_data)
#     print(json_output)

#     print ("Start processing data for Day grouping. ")
#     processed_data = process_day_data(df, custom_encoding_mapping)
#     json_output = output_json(processed_data)
#     print(json_output)

# main()


In [ ]:
# @title Plot the graph

import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import mpl_tc_fonts
import pytz

mpl_tc_fonts.set_font("Noto Sans CJK TC")

# Assuming 'time' is a regular column (not the index)
data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time'])

def save_plot(plt, today, symbol_code, end_date):
    # Save the plot in PDF format
    TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d')
    file_name_predict_with = symbol_code
    plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")


def plot_actual_vs_predicted(data_with_custom_features, today, symbol_code, end_date):

    # data_with_custom_features = data_with_custom_features_.copy().dropna(subset=['Close'], inplace=True)
    # Assuming 'time' is the index of your DataFrame
    plt.figure(figsize=(420, 60))
    # TODO
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Close'], label='Actual Close', marker='o')
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Predicted_Close'], label='Predicted Close', marker='o', linestyle='dashed')

    plt.title('Actual vs Predicted Close Values')
    plt.xlabel('Time')
    plt.ylabel('Close Values')
    plt.legend()
    plt.grid(True)
    plt.show()
    save_plot(today, symbol_code, end_date)
    # # Save the plot in PDF format
    # TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d')
    # file_name_predict_with = symbol_code
    # plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")

DATE_TIME = 'time'

def decode_and_annotate(row, feature, label_encoder, custom_encoding_mapping, ax, weight='normal', color='black', offset=55):
    try:

      decoded_value = next(key for key, value in custom_encoding_mapping.items() if value == label_encoder.inverse_transform([row[feature]]))
      ax.annotate(f"{feature}: {decoded_value}", (row['time'], row['Predicted_Close']),
                textcoords="offset points", xytext=(5, offset), rotation=90, ha='center',
                weight=weight, color=color)
    except Exception as ex:
      print(f"error {feature} {ex}")


def plot_actual_vs_predicted_vLine(input_data, today, symbol_code, end_date):

    data_with_custom_features = input_data.copy()

    # Assuming 'time' is a column in your DataFrame with datetime values in Asia/Shanghai timezone
    # data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time']).dt.tz_localize('Asia/Shanghai')


    # Assuming 'time' is the index of your DataFrame
    plt.figure(figsize=(520, 30))

    # data_with_custom_features.dropna(subset=['Close'], inplace=True)
    # data_with_custom_features['Close'] = data_with_custom_features['Close'].fillna(data_with_custom_features_['Close'].mean())


    data_close = data_with_custom_features.copy()
    data_close.dropna(subset=['Close', 'time'], inplace=True)
    # data_with_custom_features.dropna(inplace=True)
    data_with_custom_features['Predicted_Close'] = pd.to_numeric(data_with_custom_features['Predicted_Close'], errors='coerce')

    # Assuming 'time' is a column in your DataFrame with datetime values in Asia/Shanghai timezone
    data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time'])


    # Plot the actual and predicted close values
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Predicted_Close'], label='Predicted Close', marker='+', linestyle='dashed', color='red')
    # TODO
    plt.plot(data_close['time'], data_close['Close'], label='Actual Close', marker='o', color='blue')


    # Set x-axis ticks and labels for every 2 hours with 'yyyy.mm.dd hh mm' format
    date_format = mdates.DateFormatter('%Y-%m-%d %H:%M:%S%z')
    plt.gca().xaxis.set_major_formatter(date_format)

    plt.xticks(pd.date_range(start=data_with_custom_features['time'].min(),
                            end=data_with_custom_features['time'].max(), freq='2H'), rotation=45, ha='right', minor=False)


    print(f"ploting time range {data_with_custom_features['time'].max()} {data_with_custom_features['time'].min()}")
    # # Add vertical dotted lines for every 2 hours
    # for time_point in pd.date_range(start=data_with_custom_features['time'].min(), end=data_with_custom_features['time'].max(), freq='2H'):
    #     plt.axvline(x=time_point, color='gray', linestyle='--', linewidth=0.5)

    # Increase the number of x-axis ticks


    # num_data_points = len(data_with_custom_features[DATE_TIME])
    # num_ticks = min(270, num_data_points)  # Limiting ticks to the number of data points


    # x_ticks_indices = range(0, len(data_with_custom_features[DATE_TIME]), len(data_with_custom_features[DATE_TIME]) // num_ticks)
    # x_ticks_labels = data_with_custom_features[DATE_TIME].iloc[x_ticks_indices]

    # Add actual values next to each dot
    for index, row in data_with_custom_features.iterrows():
        plt.annotate(f"{row['Close']:.2f}", (row['time'], row['Close']),
                     textcoords="offset points", xytext=(5,-55), rotation=45, ha='center')
        plt.annotate(f"{row['Predicted_Close']:.2f}", (row['time'], row['Predicted_Close']),
                     textcoords="offset points", xytext=(5,15),rotation=45, weight='bold', ha='center')
        # decoded_value_流月 = label_encoder.inverse_transform([row['流月']])
        # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == row['流月'])
        # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == label_encoders[0].inverse_transform([row['流月']]))
        # plt.annotate(f"月:{decoded_value_流月}", (row['time'], row['Predicted_Close']),
        #              textcoords="offset points", xytext=(5,55),rotation=90, ha='center')
        # Assuming '流月' is the feature you want to decode and annotate
        decode_and_annotate(row, '-流月', label_encoders['-流月'], custom_encoding_mapping, offset = 305, ax=plt.gca())
        decode_and_annotate(row, '流年', label_encoders['流年'], custom_encoding_mapping, offset = 255, ax=plt.gca())
        decode_and_annotate(row, '流月', label_encoders['流月'], custom_encoding_mapping, offset = 205, ax=plt.gca(), weight='bold', color='red')
        decode_and_annotate(row, '-流時', label_encoders['-流時'], custom_encoding_mapping, offset = 155, ax=plt.gca())
        decode_and_annotate(row, '流日', label_encoders['流日'], custom_encoding_mapping, offset = 105, ax=plt.gca(),  color='blue')
        decode_and_annotate(row, '流時', label_encoders['流時'], custom_encoding_mapping, offset = 55, ax=plt.gca(),  color = 'blue')
# '流時',
# '流日',
# '-流時',
# '流月',
# '流年',
# '-流月',
    # Set labels and title
    plt.title('Actual vs Predicted Close Values')
    plt.xlabel('Time')
    plt.ylabel('Close Values')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Adjust y-axis limits to increase spacing
    plt.ylim(bottom=min(data_with_custom_features[['Close', 'Predicted_Close']].min()) - 1000,
             top=max(data_with_custom_features[['Close', 'Predicted_Close']].max()) + 1200)

    # Adjust spacing around the plot
    #plt.subplots_adjust(top=1.1, bottom=1)

    # Save the plot in PDF format
    TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d%hh%mm%ss')
    file_name_predict_with = symbol_code
    #mbol_code
    plt.plot()

    plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version_1.pdf", format="pdf")

    plt.show()
    # Show the plot
    plt.close()
    #save_plot(plt.gca(), today, symbol_code, end_date)



df_plot = data_with_custom_features.copy()
# Calculate the date range
start_date = today - datetime.timedelta(days=2)
end_date = today + datetime.timedelta(days=90)


start_date = pd.to_datetime(start_date).tz_localize('Asia/Hong_Kong')
end_date = pd.to_datetime(end_date).tz_localize('Asia/Hong_Kong')
# Filter the DataFrame based on the date range
df_plot = df_plot[(df_plot['time']>= start_date) & (df_plot['time'] <= end_date)]


plot_actual_vs_predicted_vLine(df_plot, today, symbol_code, end_date)

In [ ]:
# @title Plot the graph

import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import mpl_tc_fonts
import pytz

mpl_tc_fonts.set_font("Noto Sans CJK TC")

# Assuming 'time' is a regular column (not the index)
data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time'])

def save_plot(plt, today, symbol_code, end_date):
    # Save the plot in PDF format
    TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d')
    file_name_predict_with = symbol_code
    plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")


def plot_actual_vs_predicted(data_with_custom_features, today, symbol_code, end_date):

    # data_with_custom_features = data_with_custom_features_.copy().dropna(subset=['Close'], inplace=True)
    # Assuming 'time' is the index of your DataFrame
    plt.figure(figsize=(420, 60))
    # plt.plot(data_with_custom_features['time'], data_with_custom_features['Close'], label='Actual Close', marker='o')
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Predicted_Close'], label='Predicted Close', marker='o', linestyle='dashed')

    plt.title('Actual vs Predicted Close Values')
    plt.xlabel('Time')
    plt.ylabel('Close Values')
    plt.legend()
    plt.grid(True)
    plt.show()
    save_plot(today, symbol_code, end_date)
    # # Save the plot in PDF format
    # TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d')
    # file_name_predict_with = symbol_code
    # plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")

DATE_TIME = 'time'

def decode_and_annotate(row, feature, label_encoder, custom_encoding_mapping, ax, weight='normal', color='black', offset=55):
    try:

      decoded_value = next(key for key, value in custom_encoding_mapping.items() if value == label_encoder.inverse_transform([row[feature]]))
      ax.annotate(f"{feature}: {decoded_value}", (row['time'], row['Predicted_Close']),
                textcoords="offset points", xytext=(5, offset), rotation=90, ha='center',
                weight=weight, color=color)
    except Exception as ex:
      print(f"error {feature} {ex}")


def plot_actual_vs_predicted_vLine(input_data, today, symbol_code, end_date):

    data_with_custom_features = input_data.copy()

    # Assuming 'time' is a column in your DataFrame with datetime values in Asia/Shanghai timezone
    # data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time']).dt.tz_localize('Asia/Shanghai')


    # Assuming 'time' is the index of your DataFrame
    plt.figure(figsize=(320, 20))

    # data_with_custom_features.dropna(subset=['Close'], inplace=True)
    # data_with_custom_features['Close'] = data_with_custom_features['Close'].fillna(data_with_custom_features_['Close'].mean())
    data_close = data_with_custom_features.copy()
    # data_with_custom_features.dropna(inplace=True)
    data_close.dropna(subset=['Close', 'time'], inplace=True)
    data_with_custom_features['Predicted_Close'] = pd.to_numeric(data_with_custom_features['Predicted_Close'], errors='coerce')

    # Assuming 'time' is a column in your DataFrame with datetime values in Asia/Shanghai timezone
    data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time'])


    # Plot the actual and predicted close values
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Predicted_Close'], label='Predicted Close', marker='+', linestyle='dashed', color='red')
    plt.plot(data_close['time'], data_close['Close'], label='Actual Close', marker='o', color='blue')


    # Set x-axis ticks and labels for every 2 hours with 'yyyy.mm.dd hh mm' format
    date_format = mdates.DateFormatter('%Y-%m-%d %H:%M:%S%z')
    plt.gca().xaxis.set_major_formatter(date_format)

    plt.xticks(pd.date_range(start=data_with_custom_features['time'].min(),
                            end=data_with_custom_features['time'].max(), freq='2H'), rotation=45, ha='right', minor=False)


    print(f"ploting time range {data_with_custom_features['time'].max()} {data_with_custom_features['time'].min()}")
    # # Add vertical dotted lines for every 2 hours
    # for time_point in pd.date_range(start=data_with_custom_features['time'].min(), end=data_with_custom_features['time'].max(), freq='2H'):
    #     plt.axvline(x=time_point, color='gray', linestyle='--', linewidth=0.5)

    # Increase the number of x-axis ticks


    # num_data_points = len(data_with_custom_features[DATE_TIME])
    # num_ticks = min(270, num_data_points)  # Limiting ticks to the number of data points


    # x_ticks_indices = range(0, len(data_with_custom_features[DATE_TIME]), len(data_with_custom_features[DATE_TIME]) // num_ticks)
    # x_ticks_labels = data_with_custom_features[DATE_TIME].iloc[x_ticks_indices]

    # Add actual values next to each dot
    for index, row in data_with_custom_features.iterrows():
        plt.annotate(f"{row['Close']:.2f}", (row['time'], row['Close']),
                     textcoords="offset points", xytext=(5,-55), rotation=45, ha='center')
        plt.annotate(f"{row['Predicted_Close']:.2f}", (row['time'], row['Predicted_Close']),
                     textcoords="offset points", xytext=(5,15),rotation=45, weight='bold', ha='center')
        # decoded_value_流月 = label_encoder.inverse_transform([row['流月']])
        # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == row['流月'])
        # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == label_encoders[0].inverse_transform([row['流月']]))
        # plt.annotate(f"月:{decoded_value_流月}", (row['time'], row['Predicted_Close']),
        #              textcoords="offset points", xytext=(5,55),rotation=90, ha='center')
        # Assuming '流月' is the feature you want to decode and annotate
        decode_and_annotate(row, '-流月', label_encoders['-流月'], custom_encoding_mapping, offset = 305, ax=plt.gca())
        decode_and_annotate(row, '流年', label_encoders['流年'], custom_encoding_mapping, offset = 255, ax=plt.gca())
        decode_and_annotate(row, '流月', label_encoders['流月'], custom_encoding_mapping, offset = 205, ax=plt.gca(), weight='bold', color='red')
        decode_and_annotate(row, '-流時', label_encoders['-流時'], custom_encoding_mapping, offset = 155, ax=plt.gca())
        decode_and_annotate(row, '流日', label_encoders['流日'], custom_encoding_mapping, offset = 105, ax=plt.gca(),  color='blue')
        decode_and_annotate(row, '流時', label_encoders['流時'], custom_encoding_mapping, offset = 55, ax=plt.gca(),  color = 'blue')
# '流時',
# '流日',
# '-流時',
# '流月',
# '流年',
# '-流月',
    # Set labels and title
    plt.title('Actual vs Predicted Close Values')
    plt.xlabel('Time')
    plt.ylabel('Close Values')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Adjust y-axis limits to increase spacing
    plt.ylim(bottom=min(data_with_custom_features[['Close', 'Predicted_Close']].min()) - 800,
             top=max(data_with_custom_features[['Close', 'Predicted_Close']].max()) + 600)

    # Adjust spacing around the plot
    #plt.subplots_adjust(top=1.1, bottom=1)

    # Save the plot in PDF format
    TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d%hh%mm%ss')
    file_name_predict_with = symbol_code
    #mbol_code
    plt.plot()

    plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version_2.pdf", format="pdf")

    plt.show()
    # Show the plot
    plt.close()
    #save_plot(plt.gca(), today, symbol_code, end_date)

df_plot = data_with_custom_features.copy()
# Calculate the date range
start_date = today - datetime.timedelta(days=3)
end_date = today + datetime.timedelta(days=13)


start_date = pd.to_datetime(start_date).tz_localize('Asia/Hong_Kong')
end_date = pd.to_datetime(end_date).tz_localize('Asia/Hong_Kong')
# Filter the DataFrame based on the date range
df_plot = df_plot[(df_plot['time']>= start_date) & (df_plot['time'] <= end_date)]


plot_actual_vs_predicted_vLine(df_plot, today, symbol_code, end_date)

In [ ]:
import matplotlib.pyplot as plt

# Ensure 'time' is the DataFrame index or a column. If it's a column, adjust as needed.
plt.figure(figsize=(41, 5))  # Set the figure size for better readability
data_with_custom_features_= data_with_custom_features.copy()
# data_with_custom_features_.dropna(subset=['Close'], inplace=True)

# Ensure the 'time' column is in datetime format for proper plotting on the X-axis
data_with_custom_features_['time'] = pd.to_datetime(data_with_custom_features_['time'], errors='coerce')
# data_with_custom_features['Close'] = data_with_custom_features['Close'].fillna(data_with_custom_features_['Close'].mean())
data_close = data_with_custom_features_.copy()
data_close.dropna(subset=['Close', 'time'], inplace=True)
# data_with_custom_features_['Close'] = data_with_custom_features_['Close'].fillna(0)  # or .fillna(data_with_custom_features_['Close'].mean())
# data_with_custom_features_['Predicted_Close'] = data_with_custom_features_['Predicted_Close'].fillna(0)


# Plotting the 'Close' and 'Predicted_Close' values, handling NaN values gracefully
plt.plot(data_close['time'], data_close['Close'], label='Actual Close', color='blue', marker='o', linestyle='-')
plt.plot(data_with_custom_features_['time'], data_with_custom_features_['Predicted_Close'], label='Predicted Close', color='red', linestyle='--')


# # Convert Close and Predicted_Close to numeric types, coercing errors will turn invalid parsing to NaN
# data_with_custom_features_['Close'] = pd.to_numeric(data_with_custom_features_['Close'], errors='coerce')
# # data_with_custom_features_['Predicted_Close'] = pd.to_numeric(data_with_custom_features_['Predicted_Close'], errors='coerce')

# # Plotting the 'Close' values
# # plt.plot(data_with_custom_features_['time'], data_with_custom_features_['Close'], label='Actual Close', color='blue', marker='o', linestyle='-')
# # Plotting the 'Close' values
# # It will automatically ignore NaN values in 'Close'
# plt.plot(data_with_custom_features_['time'], data_with_custom_features_['Close'], label='Actual Close', color='blue', marker='o', linestyle='-')

# # Plotting the 'Predicted_Close' values
# plt.plot(data_with_custom_features_['time'], data_with_custom_features_['Predicted_Close'], label='Predicted Close', color='red', marker='', linestyle='--')


# Set x-axis ticks and labels for every 2 hours with 'yyyy.mm.dd hh mm' format
date_format = mdates.DateFormatter('%Y-%m-%d %H:%M:%S%z')
plt.gca().xaxis.set_major_formatter(date_format)

plt.xticks(pd.date_range(start=data_with_custom_features['time'].min(),
                        end=data_with_custom_features['time'].max(), freq='1M'), rotation=45, ha='right', minor=False)

# # Add vertical dotted lines for every 2 hours
# for time_point in pd.date_range(start=data_with_custom_features['time'].min(), end=data_with_custom_features['time'].max(), freq='2H'):
#     plt.axvline(x=time_point, color='gray', linestyle='--', linewidth=0.5)

# Increase the number of x-axis ticks


# num_data_points = len(data_with_custom_features[DATE_TIME])
# num_ticks = min(270, num_data_points)  # Limiting ticks to the number of data points


# x_ticks_indices = range(0, len(data_with_custom_features[DATE_TIME]), len(data_with_custom_features[DATE_TIME]) // num_ticks)
# x_ticks_labels = data_with_custom_features[DATE_TIME].iloc[x_ticks_indices]

# Add actual values next to each dot
# for index, row in data_with_custom_features.iterrows():
#     plt.annotate(f"{row['Close']:.2f}", (row['time'], row['Close']),
#                   textcoords="offset points", xytext=(5,-55), rotation=45, ha='center')
#     plt.annotate(f"{row['Predicted_Close']:.2f}", (row['time'], row['Predicted_Close']),
#                   textcoords="offset points", xytext=(5,15),rotation=45, weight='bold', ha='center')
#     # decoded_value_流月 = label_encoder.inverse_transform([row['流月']])
#     # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == row['流月'])
#     # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == label_encoders[0].inverse_transform([row['流月']]))
#     # plt.annotate(f"月:{decoded_value_流月}", (row['time'], row['Predicted_Close']),
#     #              textcoords="offset points", xytext=(5,55),rotation=90, ha='center')
#     # Assuming '流月' is the feature you want to decode and annotate
#     decode_and_annotate(row, '-流月', label_encoders['-流月'], custom_encoding_mapping, offset = 305, ax=plt.gca())
#     decode_and_annotate(row, '流年', label_encoders['流年'], custom_encoding_mapping, offset = 255, ax=plt.gca())
#     decode_and_annotate(row, '流月', label_encoders['流月'], custom_encoding_mapping, offset = 205, ax=plt.gca(), weight='bold', color='red')
#     decode_and_annotate(row, '-流時', label_encoders['-流時'], custom_encoding_mapping, offset = 155, ax=plt.gca())
#     decode_and_annotate(row, '流日', label_encoders['流日'], custom_encoding_mapping, offset = 105, ax=plt.gca(),  color='blue')
#     decode_and_annotate(row, '流時', label_encoders['流時'], custom_encoding_mapping, offset = 55, ax=plt.gca(),  color = 'blue')
# '流時',
# '流日',
# '-流時',
# '流月',
# '流年',
# '-流月',
# Set labels and title
plt.title('Full Year Chat')
plt.xlabel('Time')
plt.ylabel('Close Values')
plt.legend()
plt.grid(True)
plt.tight_layout()

# Adjust y-axis limits to increase spacing
plt.ylim(bottom=min(data_with_custom_features[['Close', 'Predicted_Close']].min()) - 1000,
          top=max(data_with_custom_features[['Close', 'Predicted_Close']].max()) + 1200)

# Adjust spacing around the plot
#plt.subplots_adjust(top=1.1, bottom=1)

# Save the plot in PDF format
TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d%hh%mm%ss')
file_name_predict_with = symbol_code
#mbol_code
plt.plot()

plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_year_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")


plt.show()
# Show the plot
plt.close()


In [ ]:
decripted_data = data_with_custom_features.copy()
for index, row in data_with_custom_features.iterrows():
  for feature in categorical_features:
        if feature in columns_to_ignore:
          continue
        if row[feature] == 0:
          continue
        if row[feature] == 1:
          continue
        # print(f"{feature} - {row[feature]}")
        label_decripted = label_encoders[feature].inverse_transform([row[feature]])
        feature_decripted = next(key for key, value in custom_encoding_mapping.items() if value == label_decripted )
        # feature_decripted = reverse_encoding_mapping.get(label_decripted, f"Unknown Encoding: {encoded_value}")
        decripted_data.at[index, feature] =  feature_decripted
        # print(f"{feature} - {feature_decripted}")
decripted_data.to_csv(f"drive/MyDrive/01_888/Prediction/feature_with_reverse_encode_{today}_version.csv")


In [ ]:
from datetime import datetime, timedelta

# 1. Get birthday
trum_birthday = datetime(1964,6,14,22,45,0) #hktime
biden_birthday = datetime(1942,11,20,20,30,0) #hktime

# 2. Get 8w
trum_birthday_8w = bazi.get_heavenly_branch_ymdh_pillars_base(trum_birthday.year,
                                                            trum_birthday.month,
                                                            trum_birthday.day,
                                                            trum_birthday.hour)
biden_birthday_8w = bazi.get_heavenly_branch_ymdh_pillars_base(biden_birthday.year,
                                                            biden_birthday.month,
                                                            biden_birthday.day,
                                                            biden_birthday.hour)
# 3. Calculate features

ppl_start_date = today - timedelta(days=20)
ppl_end_date = today + timedelta(days=120)
ppl_time_range = pd.date_range(start=ppl_start_date, end=ppl_end_date, freq='1H').union(pd.date_range(end_date, end_date + pd.DateOffset(months=12), freq='D'))
ppl_dataset = pd.DataFrame({'time': ppl_time_range})
ppl_dataset = chengseng.adding_8w_pillars(ppl_dataset)

# ppl_dataset['time'] = ppl_dataset['time'].dt.tz_localize('Asia/Hong_Kong')
ppl_dataset['本時'] = trum_birthday_8w["時"]
ppl_dataset['本日'] = trum_birthday_8w["日"]
ppl_dataset['-本時'] = trum_birthday_8w["-時"]
ppl_dataset['本月'] = trum_birthday_8w["月"]
ppl_dataset['本年'] = trum_birthday_8w["年"]
ppl_dataset['-本月'] = trum_birthday_8w["-月"]

ppl_dataset.columns.to_list()
ppl_dataset = chengseng.create_chengseng_for_dataset(ppl_dataset)
ppl_dataset = add_haap_features_to_df(ppl_dataset)
# 4. Encode features

ppl_data_with_custom_features, encoder = process_encode_data(ppl_dataset, categorical_features)

ppl_data_with_custom_features['Open'] = None
ppl_data_with_custom_features['High'] = None
ppl_data_with_custom_features['Low'] = None
ppl_data_with_custom_features['Close'] = None

ppl_future_features = ppl_data_with_custom_features.drop(['time', 'Open', 'High', "Low", "Close",
                                                  # "Volume" ,
                                                        # '流時',
                                                        # '流日',
                                                        # '-流時',
                                                        # '流月',
                                                        # '流年',
                                                        # '-流月',
                                                        '本時',
                                                        '本日',
                                                        '-本時',
                                                        '本月',
                                                        '本年',
                                                        '-本月'], axis=1)

ppl_future_predictions = mor_model.predict(ppl_future_features)
ppl_data_with_custom_features[['Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close']] = ppl_future_predictions

ppl_df_plot = ppl_data_with_custom_features.copy()
# Calculate the date range
start_date = today - datetime.timedelta(days=25)
end_date = today + datetime.timedelta(days=90)


start_date = pd.to_datetime(start_date).tz_localize('Asia/Hong_Kong')
end_date = pd.to_datetime(end_date).tz_localize('Asia/Hong_Kong')
# Filter the DataFrame based on the date range
df_plot = df_plot[(df_plot['time']>= start_date) & (df_plot['time'] <= end_date)]


plot_actual_vs_predicted_vLine(ppl_df_plot, today, symbol_code, end_date)


In [ ]:
# @title Plot the graph

import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import mpl_tc_fonts
import pytz
import mplfinance as mpf

mpl_tc_fonts.set_font("Noto Sans CJK TC")

# Assuming 'time' is a regular column (not the index)
data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time'])

def save_plot(plt, today, symbol_code, end_date):
    # Save the plot in PDF format
    TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d')
    file_name_predict_with = symbol_code
    plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")


def plot_actual_vs_predicted(data_with_custom_features, today, symbol_code, end_date):
    # Assuming 'time' is the index of your DataFrame
    plt.figure(figsize=(420, 60))
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Close'], label='Actual Close', marker='o')
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Predicted_Close'], label='Predicted Close', marker='o', linestyle='dashed')

    plt.title('Actual vs Predicted Close Values')
    plt.xlabel('Time')
    plt.ylabel('Close Values')
    plt.legend()
    plt.grid(True)
    plt.show()
    save_plot(today, symbol_code, end_date)
    # # Save the plot in PDF format
    # TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d')
    # file_name_predict_with = symbol_code
    # plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")

DATE_TIME = 'time'

def decode_and_annotate(row, feature, label_encoder, custom_encoding_mapping, ax, weight='normal', color='black', offset=55):
    try:

      decoded_value = next(key for key, value in custom_encoding_mapping.items() if value == label_encoder.inverse_transform([row[feature]]))
      ax.annotate(f"{feature}: {decoded_value}", (row['time'], row['Predicted_Close']),
                textcoords="offset points", xytext=(5, offset), rotation=90, ha='center',
                weight=weight, color=color)
    except Exception as ex:
      print(f"error {feature} {ex}")


def plot_actual_vs_predicted_vLine(input_data, today, symbol_code, end_date):

    data_with_custom_features = input_data.copy()

    # Assuming 'time' is a column in your DataFrame with datetime values in Asia/Shanghai timezone
    # data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time']).dt.tz_localize('Asia/Shanghai')

    # Assuming 'time' is a column in your DataFrame with datetime values in Asia/Shanghai timezone
    data_with_custom_features['time'] = pd.to_datetime(data_with_custom_features['time'])

    # Assuming 'time' is the index of your DataFrame
    plt.figure(figsize=(380, 30))

    # Plot the actual and predicted close values
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Close'], label='Actual Close', marker='o', color='blue')
    plt.plot(data_with_custom_features['time'], data_with_custom_features['Predicted_Close'], label='Predicted Close', marker='+', linestyle='dashed', color='red')


    # Set x-axis ticks and labels for every 2 hours with 'yyyy.mm.dd hh mm' format
    date_format = mdates.DateFormatter('%Y-%m-%d %H:%M:%S%z')
    plt.gca().xaxis.set_major_formatter(date_format)

    plt.xticks(pd.date_range(start=data_with_custom_features['time'].min(),
                            end=data_with_custom_features['time'].max(), freq='2H'), rotation=45, ha='right', minor=False)

    # # Add vertical dotted lines for every 2 hours
    # for time_point in pd.date_range(start=data_with_custom_features['time'].min(), end=data_with_custom_features['time'].max(), freq='2H'):
    #     plt.axvline(x=time_point, color='gray', linestyle='--', linewidth=0.5)

      # Increase the number of x-axis ticks
    num_ticks = 150  # Choose the number of ticks you want
    x_ticks_indices = range(0, len(data_with_custom_features[DATE_TIME]), len(data_with_custom_features[DATE_TIME]) // num_ticks)
    x_ticks_labels = data_with_custom_features[DATE_TIME].iloc[x_ticks_indices]

    # Add actual values next to each dot
    for index, row in data_with_custom_features.iterrows():
        plt.annotate(f"{row['Close']:.2f}", (row['time'], row['Close']),
                     textcoords="offset points", xytext=(5,-55), rotation=45, ha='center')
        plt.annotate(f"{row['Predicted_Close']:.2f}", (row['time'], row['Predicted_Close']),
                     textcoords="offset points", xytext=(5,15),rotation=45, weight='bold', ha='center')
        # decoded_value_流月 = label_encoder.inverse_transform([row['流月']])
        # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == row['流月'])
        # decoded_value_流月 = next(key for key, value in custom_encoding_mapping.items() if value == label_encoders[0].inverse_transform([row['流月']]))
        # plt.annotate(f"月:{decoded_value_流月}", (row['time'], row['Predicted_Close']),
        #              textcoords="offset points", xytext=(5,55),rotation=90, ha='center')
        # Assuming '流月' is the feature you want to decode and annotate
        decode_and_annotate(row, '-流月', label_encoders['-流月'], custom_encoding_mapping, offset = 305, ax=plt.gca())
        decode_and_annotate(row, '流年', label_encoders['流年'], custom_encoding_mapping, offset = 255, ax=plt.gca())
        decode_and_annotate(row, '流月', label_encoders['流月'], custom_encoding_mapping, offset = 205, ax=plt.gca(), weight='bold', color='red')
        decode_and_annotate(row, '-流時', label_encoders['-流時'], custom_encoding_mapping, offset = 155, ax=plt.gca())
        decode_and_annotate(row, '流日', label_encoders['流日'], custom_encoding_mapping, offset = 105, ax=plt.gca(),  color='blue')
        decode_and_annotate(row, '流時', label_encoders['流時'], custom_encoding_mapping, offset = 55, ax=plt.gca(),  color = 'blue')




# '流時',
# '流日',
# '-流時',
# '流月',
# '流年',
# '-流月',
    # Set labels and title
    plt.title('Actual vs Predicted Close Values')
    plt.xlabel('Time')
    plt.ylabel('Close Values')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Adjust y-axis limits to increase spacing
    plt.ylim(bottom=min(data_with_custom_features[['Close', 'Predicted_Close']].min()) - 1000,
             top=max(data_with_custom_features[['Close', 'Predicted_Close']].max()) + 1200)

    # Adjust spacing around the plot
    #plt.subplots_adjust(top=1.1, bottom=1)

    # Save the plot in PDF format
    TARGET_COLUMN = "Ken" + today.strftime('%Y-%m-%d%hh%mm%ss')
    file_name_predict_with = symbol_code
    #mbol_code
    plt.plot()

    plt.savefig(f"drive/MyDrive/01_888/Prediction/prediction_hourly_{TARGET_COLUMN}_{end_date}_{file_name_predict_with}_version.pdf", format="pdf")

    plt.show()
    # Show the plot
    plt.close()
    #save_plot(plt.gca(), today, symbol_code, end_date)

def plot_actual_vs_predicted_candle(input_data, today, symbol_code, end_date):
    """
    Plot actual vs predicted candlestick patterns for a given date range.

    Parameters:
        input_data (DataFrame): Input data containing historical price data.
        today (str): Date for which you want to compare actual and predicted candlestick patterns (format: 'YYYY-MM-DD').
        symbol_code (str): Symbol code for the asset.
        end_date (str): End date of the date range (format: 'YYYY-MM-DD').
    """
    # Filter data for the specified date range
    filtered_data = input_data.copy()
    # Assuming 'time' is the index of your DataFrame
    # Set the size of the figure
    plt.figure(figsize=(1200, 6))


    # Extract actual and predicted data for the specified date
    actual_data = filtered_data['Close']
    predicted_data = filtered_data[['Predicted_Open', 'Predicted_High', 'Predicted_Low', 'Predicted_Close']]  # Replace this with your predicted data
    predicted_data.columns = ['Open', 'High', 'Low', 'Close']  # Rename columns for compatibility with mplfinance
    predicted_data.index = filtered_data['time']
    # Plot actual candlestick chart
    fig, ax = plt.subplots(figsize=(150, 20))
    # plt.plot(filtered_data['time'], filtered_data['Close'], label='Actual Close', marker='o', color='blue')
    # Increase the number of x-axis ticks
    # num_ticks = 150  # Choose the number of ticks you want
    # x_ticks_indices = range(0, len(data_with_custom_features[DATE_TIME]), len(data_with_custom_features[DATE_TIME]) // num_ticks)
    # x_ticks_labels = data_with_custom_features[DATE_TIME].iloc[x_ticks_indices]

    # plt.xticks(pd.date_range(start=filtered_data['time'].min(),
    #                         end=filtered_data['time'].max(), freq='2H'), rotation=45, ha='right', minor=False)
    # Overlay predicted candlestick chart
    predicted_plot = mpf.plot(predicted_data, type='candle', style='starsandstripes', ax=ax)

    # # Check if plots were successfully created before adding title
    # if predicted_plot is not None:
    #     plt.suptitle(f'Actual vs Predicted Candlestick Chart - {symbol_code}')
    #     plt.show()
    # else:
    #     print("Error: Failed to create plots.")

    # Set x-axis ticks every two hours
    # tick_locations = pd.date_range(start=filtered_data['time'].min(), end=filtered_data['time'].max(), freq='2H')
    # tick_labels = [tick.strftime('%Y-%m-%d %H:%M:%S') for tick in tick_locations]
    # plt.xticks(tick_locations, tick_labels)

    # Add title
    plt.title(f'Actual vs Predicted Candlestick Chart - {symbol_code}')

    plt.show()

df_plot = data_with_custom_features.copy()
# Calculate the date range
start_date = today - datetime.timedelta(days=25)
end_date = today + datetime.timedelta(days=90)


start_date = pd.to_datetime(start_date).tz_localize('Asia/Hong_Kong')
end_date = pd.to_datetime(end_date).tz_localize('Asia/Hong_Kong')
# Filter the DataFrame based on the date range
df_plot = df_plot[(df_plot['time']>= start_date) & (df_plot['time'] <= end_date)]


# plot_actual_vs_predicted_vLine(df_plot, today, symbol_code, end_date)
plot_actual_vs_predicted_candle(df_plot, today, symbol_code, end_date)